
## Ejercicio 3

Para resolver este ejercicio utilice un modelo de red neuronal convolucional que reconozca la cantidad de dedos extendidos en cada mano de las imágenes del dataset **“Fingers”**.

<div style="text-align: center;">
    <img src='../../images/p4-ej10.png' width="40%">
</div>

La versión original de este de estas imágenes se encuentra en https://www.kaggle.com/koryakinp/fingers.

Puede hallar una versión reducida de estas imágenes en el Moodle del curso, en la misma sección donde se encuentra este enunciado de práctica. También encontrará allí ejemplos sobre cómo cargar estas imágenes y cómo procesarlas con una red neuronal convolucional.

### a)

Entrene y pruebe un modelo utilizando los datos de las carpetas **test** y **train**, midiendo **accuracy**.

### b)

Genere una versión del **dataset** para **test** agregando transformaciones al azar sobre imágenes originales. Haga **rotaciones** entre -45° y 45°, repita el test y mida el accuracy.

### c)

Genere una versión del **dataset train** como en b) y repita el entrenamiento y prueba del punto a) con los datasets modificados.